# Load a prepared SAMPL6 system


This notebook downloads a SAMPL6 challenge system as an XML-serialized OpenMM System, creates an `Interchange` from it, and then computes the energy of the starting coordinates with GROMACS.

In [1]:
from openff.interchange import Interchange
from openff.interchange.drivers import get_summary_data

In [2]:
!wget -O complex.pdb https://raw.githubusercontent.com/samplchallenges/SAMPL6/c661d3985af7fa0ba8c64a1774cfb2363cd31bda/host_guest/SAMPLing/CB8-G3-3/PDB/complex.pdb
!wget -O complex.xml https://raw.githubusercontent.com/samplchallenges/SAMPL6/c661d3985af7fa0ba8c64a1774cfb2363cd31bda/host_guest/SAMPLing/CB8-G3-3/OPENMM/complex.xml

--2025-09-30 00:14:45--  https://raw.githubusercontent.com/samplchallenges/SAMPL6/c661d3985af7fa0ba8c64a1774cfb2363cd31bda/host_guest/SAMPLing/CB8-G3-3/PDB/complex.pdb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.


HTTP request sent, awaiting response... 

200 OK
Length: 537508 (525K) [text/plain]
Saving to: ‘complex.pdb’

complex.pdb         100%[===================>] 524.91K  --.-KB/s    in 0.03s   

2025-09-30 00:14:45 (18.7 MB/s) - ‘complex.pdb’ saved [537508/537508]



--2025-09-30 00:14:46--  https://raw.githubusercontent.com/samplchallenges/SAMPL6/c661d3985af7fa0ba8c64a1774cfb2363cd31bda/host_guest/SAMPLing/CB8-G3-3/OPENMM/complex.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.


HTTP request sent, awaiting response... 

200 OK
Length: 1942552 (1.9M) [text/plain]
Saving to: ‘complex.xml’

complex.xml           0%[                    ]       0  --.-KB/s               

complex.xml         100%[===================>]   1.85M  --.-KB/s    in 0.04s   

2025-09-30 00:14:46 (41.7 MB/s) - ‘complex.xml’ saved [1942552/1942552]



In [3]:
import openmm
import openmm.app

pdb_object = openmm.app.PDBFile("complex.pdb")

topology = pdb_object.topology
positions = pdb_object.positions

system = openmm.XmlSerializer.deserialize(open("complex.xml").read())

# Remove force objects that are not stored in Interchange and do not affect the energy
for force_index in range(system.getNumForces())[::-1]:
    if isinstance(
        system.getForce(force_index),
        openmm.MonteCarloBarostat | openmm.CMMotionRemover,
    ):
        system.removeForce(force_index)

In [4]:
imported = Interchange.from_openmm(
    system=system,
    topology=pdb_object.topology,
    positions=pdb_object.getPositions(),
)

# Residue numbers in this PDB start at 0, but GROMACS requires that
# residue numbers start at 1, so modify them in-place
for atom in imported.topology.atoms:
    atom.metadata["residue_number"] = str(int(atom.metadata["residue_number"]) + 1)

In [5]:
# Interchange.to_amber is not yet optimized for biopolymer systems
summary = get_summary_data(imported, _engines=["GROMACS", "OpenMM"])
summary

/home/runner/micromamba/envs/openff-docs-examples/lib/python3.11/site-packages/openff/interchange/components/mdconfig.py:502: UserWarning: Ambiguous failure while processing constraints. Constraining h-bonds as a stopgap.
  warnings.warn(


,Bond,Angle,Torsion,vdW,Electrostatics,RBTorsion
OpenMM,181.5107,909.194443,516.769674,12591.934473,-109467.404433,NaN
GROMACS,181.5103,909.194092,516.769653,12591.974281,-109465.802429,0.0
